## 情報をとってみる

In [1]:
import requests
from bs4 import BeautifulSoup

url = 'https://global.sbisec.co.jp/invest/us/stock/NVDA?resource=price&searchType=include'

res = requests.get(url)

soup = BeautifulSoup(res.text, 'html.parser')

# print(soup)

## selenium使ってみる

In [1]:
import os
import time
import datetime
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

In [4]:
from key import user_name, password

ACCOUNT = user_name
PASSWORD = password
# url = 'https://global.sbisec.co.jp/home'
url = 'https://www.sbisec.co.jp/ETGate'

def connect_sbi(ACCOUNT, PASSWORD, url):
    options = Options()
    
    driver = webdriver.Firefox(options=options)
    driver.get(url)

    # ログイン処理
    driver.find_element(By.NAME, 'user_id').send_keys(ACCOUNT)
    driver.find_element(By.NAME, 'user_password').send_keys(PASSWORD)
    driver.find_element(By.NAME, 'ACT_login').click()
    
    return driver

print(connect_sbi(user_name, password, url))

<selenium.webdriver.firefox.webdriver.WebDriver (session="0282f8e9-7e3b-4808-9053-4ba6bd5a76b3")>


<Response [200]>
aaaaaaaaaa
